In [ ]:
#/FileStore/tables/movie_metadata.tsv
# File uploaded to /FileStore/tables/plot_summaries.txt

In [ ]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
plot_summaries = sc.textFile("/FileStore/tables/plot_summaries.txt")
movies_metadata = sc.textFile("/FileStore/tables/movie_metadata.tsv")

In [ ]:
import nltk

In [ ]:
import math
from pyspark.sql.functions import *
import pyspark.sql.functions as func

In [ ]:
plot_summaries = plot_summaries.map(lambda x: x.lower())
#plot_summaries.collect()

In [ ]:
plot_summaries = plot_summaries.map(lambda x: x.split("\t"))
#plot_summaries.collect()

In [ ]:
movies_metadata = movies_metadata.map(lambda x: x.lower())
#movies_metadata.collect()

In [ ]:
movies_metadata = movies_metadata.map(lambda x: (x.split("\t")[0], x.split("\t")[2]))
#movies_metadata.collect()

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Out[9]: True

In [ ]:
list_of_stopwords = stopwords.words("english")

In [ ]:
count = plot_summaries.count()
count

Out[11]: 42306

In [ ]:
plotSummaries1 = plot_summaries.flatMap(lambda x: [((x[0], i), 1) for i in x[1].split()])
#plotSummaries1.collect()

In [ ]:
plotSummaries1 = plotSummaries1.filter(lambda x: x[0][1] not in list_of_stopwords)

In [ ]:
#plotSummaries1.take(50)

In [ ]:
plotSummariesReduced = plotSummaries1.reduceByKey(lambda x,y: x+y)

In [ ]:
#plotSummariesReduced.collect()

In [ ]:
plotSummariesTF = plotSummariesReduced.map(lambda x: (x[0][1], (x[0][0], x[1])))

In [ ]:
#plotSummariesTF.take(25)

In [ ]:
plotSummariesIDF1 = plotSummariesReduced.map(lambda x: (x[0][1],(x[0][0],x[1],1)))

In [ ]:
plotSummariesIDF1.take(25)

Out[20]: [('([[brian', ('2162986', 1, 1)),
 ('appears', ('10017886', 1, 1)),
 ('terrorists.', ('33222661', 1, 1)),
 ('since', ('30695973', 1, 1)),
 ('convinces', ('6048468', 1, 1)),
 ('please,', ('34372982', 1, 1)),
 ('suicide,', ('4977595', 1, 1)),
 ('engineer', ('633692', 2, 1)),
 ('regain', ('5526916', 1, 1)),
 ('leaving', ('3956101', 1, 1)),
 ('parenting;', ('21500407', 1, 1)),
 ('together', ('14711494', 1, 1)),
 ('finger,', ('8454631', 1, 1)),
 ('high', ('33059077', 2, 1)),
 ('conclusion', ('33059077', 1, 1)),
 ('course,', ('12576808', 2, 1)),
 ('meanwhile,', ('5846938', 1, 1)),
 ('commemoration', ('2769852', 1, 1)),
 ('jyoti', ('5760640', 2, 1)),
 ('jealous.', ('3694594', 1, 1)),
 ('easy.', ('1597802', 1, 1)),
 ('amassed', ('9756837', 1, 1)),
 ('behind.', ('25092646', 1, 1)),
 ('starting', ('1576720', 1, 1)),
 ('fantasy', ('550486', 1, 1))]

In [ ]:
plotSummariesIDF2 = plotSummariesIDF1.map(lambda x: (x[0],x[1][2]))

In [ ]:
#plotSummariesIDF2.take(25)

In [ ]:
plotSummariesIDFreduced = plotSummariesIDF2.reduceByKey(lambda x,y: x+y)

In [ ]:
plotSummariesIDFreduced.take(25)

Out[24]: [('getting', 3199),
 ('however;', 12),
 ('leaves', 5811),
 ('burning', 594),
 ('bank,', 218),
 ('richards', 45),
 ('increase', 190),
 ('conflicts', 158),
 ('sickly', 53),
 ('survivors', 521),
 ('jewels,', 35),
 ('beneath', 419),
 ('secret', 2555),
 ('religion', 106),
 ('everywhere,', 42),
 ('loading', 51),
 ('marty', 92),
 ('using', 3526),
 ('kills', 3812),
 ('let', 2315),
 ('know', 3453),
 ('town,', 1148),
 ('policeman', 404),
 ('fall', 3088),
 ('uses', 2975)]

In [ ]:
import math

In [ ]:
plotSummariesIDF3 = plotSummariesIDFreduced.map(lambda x: (x[0],math.log10(count/x[1])))

In [ ]:
plotSummariesIDF3.take(15)

Out[27]: [('getting', 1.1213877249765791),
 ('however;', 3.5472207190130614),
 ('leaves', 0.8621510896219129),
 ('burning', 1.8526155200794925),
 ('bank,', 2.2879454714560814),
 ('richards', 2.9731894512853425),
 ('increase', 2.347648364107857),
 ('conflicts', 2.4277448781062634),
 ('sickly', 2.902126095459897),
 ('survivors', 1.9095642417611616),
 ('jewels,', 3.0823339207104103),
 ('beneath', 2.004187942094391),
 ('secret', 1.2190110605899545),
 ('religion', 2.6010960997959156),
 ('everywhere,', 3.0031526746627857)]

In [ ]:
plotSummaries_tfidf_joined = plotSummariesTF.join(plotSummariesIDF3)

In [ ]:
plotSummaries_tfidf_joined.take(15)

Out[29]: [('says,', (('12233591', 1), 1.964589279523425)),
 ('says,', (('29260857', 1), 1.964589279523425)),
 ('says,', (('11418316', 1), 1.964589279523425)),
 ('says,', (('2161511', 1), 1.964589279523425)),
 ('says,', (('6602433', 1), 1.964589279523425)),
 ('says,', (('3117366', 1), 1.964589279523425)),
 ('says,', (('4152250', 1), 1.964589279523425)),
 ('says,', (('6816597', 1), 1.964589279523425)),
 ('says,', (('8776448', 1), 1.964589279523425)),
 ('says,', (('6601066', 1), 1.964589279523425)),
 ('says,', (('545795', 1), 1.964589279523425)),
 ('says,', (('19476185', 1), 1.964589279523425)),
 ('says,', (('1278961', 1), 1.964589279523425)),
 ('says,', (('23325348', 1), 1.964589279523425)),
 ('says,', (('875305', 1), 1.964589279523425))]

In [ ]:
plotSummaries_tfIDF = plotSummaries_tfidf_joined.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1])))

In [ ]:
#plotSummaries_tfIDF.take(15)

In [ ]:
singleTermSearch = "women"

In [ ]:
searched = plotSummaries_tfIDF.filter(lambda x: x[1][0]==singleTermSearch).sortBy(lambda x: -x[1][3])

In [ ]:
#searched.take(10)

In [ ]:
movies_metadata

Out[35]: PythonRDD[48] at RDD at PythonRDD.scala:58

In [ ]:
searchedTermResults = searched.join(movies_metadata)

In [ ]:
#searchedTermResults.take(10)

In [ ]:
############################### Question 2 part 2

In [ ]:
# multiple term search

multipleTermQuery = " Wall Street Mark romantic relationship with actress Anna due to Vermeer painting "

In [ ]:
multipleTermQuery = multipleTermQuery.lower()

In [ ]:
multipleTermQuery = multipleTermQuery.split(' ')

In [ ]:
multipleTermQuery

Out[42]: ['',
 'wall',
 'street',
 'mark',
 'romantic',
 'relationship',
 'with',
 'actress',
 'anna',
 'due',
 'to',
 'vermeer',
 'painting',
 '']

In [ ]:
mtq_rdd = sc.parallelize(list(multipleTermQuery))

In [ ]:
mtq_rdd = mtq_rdd.filter(lambda x: x not in list_of_stopwords)
mtq_rdd.collect()

Out[44]: ['',
 'wall',
 'street',
 'mark',
 'romantic',
 'relationship',
 'actress',
 'anna',
 'due',
 'vermeer',
 'painting',
 '']

In [ ]:
#mtq_rdd.collect()
qcount = mtq_rdd.count()
qcount

Out[45]: 12

In [ ]:
mtq_rdd = mtq_rdd.map(lambda x: (x,1))
mtq_rdd = mtq_rdd.reduceByKey(lambda x,y:(x+y))

In [ ]:
mtq_rdd.collect()

Out[47]: [('', 2),
 ('romantic', 1),
 ('wall', 1),
 ('street', 1),
 ('actress', 1),
 ('anna', 1),
 ('painting', 1),
 ('vermeer', 1),
 ('mark', 1),
 ('relationship', 1),
 ('due', 1)]

In [ ]:
plotSummaries_tfIDF.take(5)

Out[48]: [('12233591', ('says,', 1, 1.964589279523425, 1.964589279523425)),
 ('29260857', ('says,', 1, 1.964589279523425, 1.964589279523425)),
 ('11418316', ('says,', 1, 1.964589279523425, 1.964589279523425)),
 ('2161511', ('says,', 1, 1.964589279523425, 1.964589279523425)),
 ('6602433', ('says,', 1, 1.964589279523425, 1.964589279523425))]

In [ ]:
keys = mtq_rdd.keys().collect()
keys

Out[49]: ['',
 'romantic',
 'wall',
 'street',
 'actress',
 'anna',
 'painting',
 'vermeer',
 'mark',
 'relationship',
 'due']

In [ ]:
idfs = plotSummaries_tfIDF.filter(lambda x: (x[1][0] in keys))


In [ ]:
idfs = idfs.map(lambda x: (x[1][0], x[1][2]))
idfs.collect()

Out[51]: [('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),
 ('mark', 1.8797677661231074),

In [ ]:
idfss = idfs.reduceByKey(lambda x,y:x)

In [ ]:
mtq_tfidf = mtq_rdd.join(idfss)
mtq_tfidf.collect()

Out[53]: [('romantic', (1, 1.5336813203765869)),
 ('wall', (1, 1.7309794190212782)),
 ('street', (1, 1.4999458516288817)),
 ('actress', (1, 1.8813271734786285)),
 ('mark', (1, 1.8797677661231074)),
 ('relationship', (1, 1.1038273323695094)),
 ('anna', (1, 2.125342702842935)),
 ('painting', (1, 2.092375859004551)),
 ('vermeer', (1, 3.9274319607246673)),
 ('due', (1, 1.0913611518095256))]

In [ ]:
mtq_tfidf = mtq_tfidf.map(lambda x: (x[0],(x[1][0]/qcount,x[1][1])))
mtq_tfidf.collect()

Out[54]: [('romantic', (0.08333333333333333, 1.5336813203765869)),
 ('wall', (0.08333333333333333, 1.7309794190212782)),
 ('street', (0.08333333333333333, 1.4999458516288817)),
 ('actress', (0.08333333333333333, 1.8813271734786285)),
 ('mark', (0.08333333333333333, 1.8797677661231074)),
 ('relationship', (0.08333333333333333, 1.1038273323695094)),
 ('anna', (0.08333333333333333, 2.125342702842935)),
 ('painting', (0.08333333333333333, 2.092375859004551)),
 ('vermeer', (0.08333333333333333, 3.9274319607246673)),
 ('due', (0.08333333333333333, 1.0913611518095256))]

In [ ]:
# User query with normalized tf, idf, tf-idf

In [ ]:
mtq_tf_idf = mtq_tfidf.map(lambda x:(x[0],(x[1][0],x[1][1],x[1][0]*x[1][1])))
mtq_tf_idf.collect()

Out[56]: [('romantic', (0.08333333333333333, 1.5336813203765869, 0.1278067766980489)),
 ('wall', (0.08333333333333333, 1.7309794190212782, 0.14424828491843983)),
 ('street', (0.08333333333333333, 1.4999458516288817, 0.12499548763574014)),
 ('actress', (0.08333333333333333, 1.8813271734786285, 0.15677726445655238)),
 ('mark', (0.08333333333333333, 1.8797677661231074, 0.15664731384359226)),
 ('relationship',
  (0.08333333333333333, 1.1038273323695094, 0.09198561103079245)),
 ('anna', (0.08333333333333333, 2.125342702842935, 0.1771118919035779)),
 ('painting', (0.08333333333333333, 2.092375859004551, 0.17436465491704592)),
 ('vermeer', (0.08333333333333333, 3.9274319607246673, 0.3272859967270556)),
 ('due', (0.08333333333333333, 1.0913611518095256, 0.0909467626507938))]

In [ ]:
plotSummaries_tfIDF.take(5)

Out[57]: [('12233591', ('says,', 1, 1.964589279523425, 1.964589279523425)),
 ('29260857', ('says,', 1, 1.964589279523425, 1.964589279523425)),
 ('11418316', ('says,', 1, 1.964589279523425, 1.964589279523425)),
 ('2161511', ('says,', 1, 1.964589279523425, 1.964589279523425)),
 ('6602433', ('says,', 1, 1.964589279523425, 1.964589279523425))]

In [ ]:
pkeys = plotSummaries_tfIDF.countByKey().items()
pkeys
pkeys = sc.parallelize(pkeys)
pkeys

Out[58]: ParallelCollectionRDD[83] at readRDDFromInputStream at PythonRDD.scala:435

In [ ]:
new_tf_idf = plotSummaries_tfIDF.join(pkeys)
new_tf_idf.take(5)

Out[59]: [('11418316', (('says,', 1, 1.964589279523425, 1.964589279523425), 74)),
 ('11418316', (('egg;', 1, 4.325371969396705, 4.325371969396705), 74)),
 ('11418316', (('straw', 1, 2.6396302307944413, 2.6396302307944413), 74)),
 ('11418316', (('nearby', 1, 1.3891096274932273, 1.3891096274932273), 74)),
 ('11418316', (('watercooler,', 1, 4.626401965060686, 4.626401965060686), 74))]

In [ ]:
new_tfidf = new_tf_idf.map(lambda x: (x[0],(x[1][0][0],x[1][0][1]/x[1][1],x[1][0][2],x[1][0][3])))
new_tfidf.take(5)

Out[60]: [('11418316',
  ('says,', 0.013513513513513514, 1.964589279523425, 1.964589279523425)),
 ('11418316',
  ('egg;', 0.013513513513513514, 4.325371969396705, 4.325371969396705)),
 ('11418316',
  ('straw', 0.013513513513513514, 2.6396302307944413, 2.6396302307944413)),
 ('11418316',
  ('nearby', 0.013513513513513514, 1.3891096274932273, 1.3891096274932273)),
 ('11418316',
  ('watercooler,',
   0.013513513513513514,
   4.626401965060686,
   4.626401965060686))]

In [ ]:
new_tfidf = new_tfidf.map(lambda x: (x[1][0],(x[0],x[1][1],x[1][2],x[1][1]*x[1][2])))
new_tfidf.take(5)


Out[61]: [('says,',
  ('11418316', 0.013513513513513514, 1.964589279523425, 0.026548503777343583)),
 ('egg;',
  ('11418316', 0.013513513513513514, 4.325371969396705, 0.058450972559414935)),
 ('straw',
  ('11418316',
   0.013513513513513514,
   2.6396302307944413,
   0.035670678794519475)),
 ('nearby',
  ('11418316',
   0.013513513513513514,
   1.3891096274932273,
   0.018771751722881452)),
 ('watercooler,',
  ('11418316', 0.013513513513513514, 4.626401965060686, 0.06251894547379305))]

In [ ]:
doc_tfidf = new_tfidf.filter(lambda x: x[0] in keys)

In [ ]:
doc_tfidf.collect()

Out[63]: [('mark',
  ('311935', 0.06439393939393939, 1.8797677661231074, 0.12104565160641222)),
 ('relationship',
  ('311935', 0.003787878787878788, 1.1038273323695094, 0.004181164137763293)),
 ('due',
  ('311935', 0.007575757575757576, 1.0913611518095256, 0.008267887513708529)),
 ('street',
  ('2390149',
   0.002631578947368421,
   1.4999458516288817,
   0.0039472259253391625)),
 ('due',
  ('5198820',
   0.001941747572815534,
   1.0913611518095256,
   0.0021191478675913118)),
 ('due',
  ('1826422',
   0.0029498525073746312,
   1.0913611518095256,
   0.003219354430116595)),
 ('wall',
  ('23322195',
   0.0034129692832764505,
   1.7309794190212782,
   0.005907779587103338)),
 ('romantic',
  ('3497365',
   0.0022172949002217295,
   1.5336813203765869,
   0.0034006237702363345)),
 ('mark',
  ('17981877',
   0.0021645021645021645,
   1.8797677661231074,
   0.0040687613985348646)),
 ('romantic',
  ('17752771',
   0.0022624434389140274,
   1.5336813203765869,
   0.0034698672406710115)),
 ('st

In [ ]:
doc_tfidf = doc_tfidf.map(lambda x: (x[0],(x[1][0],x[1][3])))
doc_tfidf.collect()

Out[64]: [('mark', ('311935', 0.12104565160641222)),
 ('relationship', ('311935', 0.004181164137763293)),
 ('due', ('311935', 0.008267887513708529)),
 ('street', ('2390149', 0.0039472259253391625)),
 ('due', ('5198820', 0.0021191478675913118)),
 ('due', ('1826422', 0.003219354430116595)),
 ('wall', ('23322195', 0.005907779587103338)),
 ('romantic', ('3497365', 0.0034006237702363345)),
 ('mark', ('17981877', 0.0040687613985348646)),
 ('romantic', ('17752771', 0.0034698672406710115)),
 ('street', ('17752771', 0.0033935426507440765)),
 ('painting', ('17752771', 0.004733882033946949)),
 ('street', ('3654231', 0.003274990942421139)),
 ('relationship', ('3654231', 0.0024101033457849547)),
 ('due', ('1066956', 0.004938285754794234)),
 ('due', ('5855933', 0.005196957865759647)),
 ('relationship', ('2396904', 0.0022435514885559135)),
 ('due', ('2396904', 0.002218213723190093)),
 ('street', ('2261684', 0.012031651216274454)),
 ('street', ('144161', 0.0035376081406341547)),
 ('romantic', ('244246

In [ ]:
mtq_tfidf = mtq_tf_idf.map(lambda x:(x[0],x[1][2]))

In [ ]:
mtq_tfidf.collect()

Out[66]: [('romantic', 0.1278067766980489),
 ('wall', 0.14424828491843983),
 ('street', 0.12499548763574014),
 ('actress', 0.15677726445655238),
 ('mark', 0.15664731384359226),
 ('relationship', 0.09198561103079245),
 ('anna', 0.1771118919035779),
 ('painting', 0.17436465491704592),
 ('vermeer', 0.3272859967270556),
 ('due', 0.0909467626507938)]

In [ ]:
joined_tfidf = doc_tfidf.join(mtq_tfidf)
joined_tfidf.collect()

Out[67]: [('romantic', (('3497365', 0.0034006237702363345), 0.1278067766980489)),
 ('romantic', (('17752771', 0.0034698672406710115), 0.1278067766980489)),
 ('romantic', (('2442464', 0.003912452347899456), 0.1278067766980489)),
 ('romantic', (('34744345', 0.0034542372080553756), 0.1278067766980489)),
 ('romantic', (('1024501', 0.00663931307522332), 0.1278067766980489)),
 ('romantic', (('3879783', 0.005381337966233638), 0.1278067766980489)),
 ('romantic', (('449352', 0.00488433541521206), 0.1278067766980489)),
 ('romantic', (('21878601', 0.01357240106527953), 0.1278067766980489)),
 ('romantic', (('23726874', 0.0073381881357731426), 0.1278067766980489)),
 ('romantic', (('28408035', 0.01518496356808502), 0.1278067766980489)),
 ('romantic', (('2570318', 0.01322139069290161), 0.1278067766980489)),
 ('romantic', (('1371113', 0.0059909426577210425), 0.1278067766980489)),
 ('romantic', (('21757059', 0.006285579181871258), 0.1278067766980489)),
 ('romantic', (('24225279', 0.006038115434553491),

In [ ]:
joined_tfidf = joined_tfidf.map(lambda x:(x[1][0][0],(x[0],x[1][0][1],x[1][1])))
joined_tfidf.collect()

Out[68]: [('3497365', ('romantic', 0.0034006237702363345, 0.1278067766980489)),
 ('17752771', ('romantic', 0.0034698672406710115, 0.1278067766980489)),
 ('2442464', ('romantic', 0.003912452347899456, 0.1278067766980489)),
 ('34744345', ('romantic', 0.0034542372080553756, 0.1278067766980489)),
 ('1024501', ('romantic', 0.00663931307522332, 0.1278067766980489)),
 ('3879783', ('romantic', 0.005381337966233638, 0.1278067766980489)),
 ('449352', ('romantic', 0.00488433541521206, 0.1278067766980489)),
 ('21878601', ('romantic', 0.01357240106527953, 0.1278067766980489)),
 ('23726874', ('romantic', 0.0073381881357731426, 0.1278067766980489)),
 ('28408035', ('romantic', 0.01518496356808502, 0.1278067766980489)),
 ('2570318', ('romantic', 0.01322139069290161, 0.1278067766980489)),
 ('1371113', ('romantic', 0.0059909426577210425, 0.1278067766980489)),
 ('21757059', ('romantic', 0.006285579181871258, 0.1278067766980489)),
 ('24225279', ('romantic', 0.006038115434553491, 0.1278067766980489)),
 ('91

In [ ]:
############ Query and Document Tf-Idf joined.
# COL 1 = Doc ID
# COL 2 = word
# COL 3 = DOC TF-IDF
# COL 4 = QUERY TF-IDF

In [ ]:
# Finding Cosine Similarity between search term and docs

In [ ]:
joined_dotproduct_docsquare = joined_tfidf.map(lambda x: (x[0],(x[1][1]*x[1][2],x[1][1]*x[1][1])))

In [ ]:
joined_dotproduct_docsquare.collect()

Out[72]: [('3497365', (0.00043462276283667237, 1.1564242026696383e-05)),
 ('17752771', (0.00044347254760031507, 1.203997866788186e-05)),
 ('2442464', (0.000500037923569743, 1.5307283374583965e-05)),
 ('34744345', (0.0004414749235120253, 1.1931754689514197e-05)),
 ('1024501', (0.0008485492036335032, 4.408047811083133e-05)),
 ('3879783', (0.0006877714597871553, 2.8958798306827588e-05)),
 ('449352', (0.0006242511657303798, 2.3856732448294768e-05)),
 ('21878601', (0.001734644832206542, 0.00018421007067680092)),
 ('23726874', (0.0009378701724370299, 5.384900511600171e-05)),
 ('28408035', (0.00194074124791425, 0.0002305831185640693)),
 ('2570318', (0.0016897833279253382, 0.00017480517185434533)),
 ('1371113', (0.0007656830704661689, 3.589139392810167e-05)),
 ('21757059', (0.0008033396149153249, 3.9508505651573356e-05)),
 ('24225279', (0.0007717120710210206, 3.6458838000993097e-05)),
 ('9179879', (0.00194074124791425, 0.0002305831185640693)),
 ('2650227', (0.0005234041816804786, 1.67712955411

In [ ]:
reduced_rdd_ = joined_dotproduct_docsquare.reduceByKey(lambda x,y:(x[0]+y[0],x[1]+y[1]))

In [ ]:
reduced_rdd_.collect()

Out[74]: [('3497365', (0.00043462276283667237, 1.1564242026696383e-05)),
 ('2442464', (0.000500037923569743, 1.5307283374583965e-05)),
 ('3879783', (0.002809373052214634, 0.00032762587284904804)),
 ('23726874', (0.0009378701724370299, 5.384900511600171e-05)),
 ('2570318', (0.0016897833279253382, 0.00017480517185434533)),
 ('2650227', (0.0006559218901449884, 1.8894409695468095e-05)),
 ('3759313', (0.0008869450952006301, 4.815991467152744e-05)),
 ('6106857', (0.0010477151326754865, 4.426965349333042e-05)),
 ('4288473', (0.000787172216084228, 2.498959416140406e-05)),
 ('981884', (0.00038969158258317935, 9.296817079519187e-06)),
 ('20089961', (0.0028711318656139363, 0.00037399293572879357)),
 ('28820585', (0.0009751983385041754, 5.822079632860749e-05)),
 ('26553611', (0.0030627322818646756, 0.0005742623028496267)),
 ('23293894', (0.0009820168240258686, 3.88917554069651e-05)),
 ('28763446', (0.002099332159194301, 8.933328336997026e-05)),
 ('499810', (0.008611841621115186, 0.0018183056189040

In [ ]:
mtq_tf_idf.collect()

Out[75]: [('romantic', (0.08333333333333333, 1.5336813203765869, 0.1278067766980489)),
 ('wall', (0.08333333333333333, 1.7309794190212782, 0.14424828491843983)),
 ('street', (0.08333333333333333, 1.4999458516288817, 0.12499548763574014)),
 ('actress', (0.08333333333333333, 1.8813271734786285, 0.15677726445655238)),
 ('mark', (0.08333333333333333, 1.8797677661231074, 0.15664731384359226)),
 ('relationship',
  (0.08333333333333333, 1.1038273323695094, 0.09198561103079245)),
 ('anna', (0.08333333333333333, 2.125342702842935, 0.1771118919035779)),
 ('painting', (0.08333333333333333, 2.092375859004551, 0.17436465491704592)),
 ('vermeer', (0.08333333333333333, 3.9274319607246673, 0.3272859967270556)),
 ('due', (0.08333333333333333, 1.0913611518095256, 0.0909467626507938))]

In [ ]:
value = mtq_tf_idf.map(lambda x: x[1][2])
value.collect()

Out[76]: [0.1278067766980489,
 0.14424828491843983,
 0.12499548763574014,
 0.15677726445655238,
 0.15664731384359226,
 0.09198561103079245,
 0.1771118919035779,
 0.17436465491704592,
 0.3272859967270556,
 0.0909467626507938]

In [ ]:
val = value.map(lambda x: math.pow(x,2)).collect()

In [ ]:
val

Out[78]: [0.01633457216994494,
 0.020807567701911396,
 0.015623871929296467,
 0.02457911065047976,
 0.024538380934412893,
 0.008461352636708245,
 0.031368622253664664,
 0.030403032884340507,
 0.10711612365362225,
 0.008271313636659821]

In [ ]:
sum1 = 0
for i in val:
    sum1+=i

In [ ]:
sum_sqroot = math.sqrt(sum1)
sum_sqroot

Out[80]: 0.5361939466751195

In [ ]:
root_rdd = reduced_rdd_.map(lambda x: (x[0],(x[1][0],math.sqrt(x[1][1]))))
root_rdd.collect()

Out[81]: [('3497365', (0.00043462276283667237, 0.0034006237702363345)),
 ('2442464', (0.000500037923569743, 0.003912452347899456)),
 ('3879783', (0.002809373052214634, 0.018100438471182072)),
 ('23726874', (0.0009378701724370299, 0.0073381881357731426)),
 ('2570318', (0.0016897833279253382, 0.01322139069290161)),
 ('2650227', (0.0006559218901449884, 0.004346770030202667)),
 ('3759313', (0.0008869450952006301, 0.006939734481342023)),
 ('6106857', (0.0010477151326754865, 0.006653544430852658)),
 ('4288473', (0.000787172216084228, 0.004998959307836388)),
 ('981884', (0.00038969158258317935, 0.0030490682313649834)),
 ('20089961', (0.0028711318656139363, 0.01933889696256727)),
 ('28820585', (0.0009751983385041754, 0.007630255325256651)),
 ('26553611', (0.0030627322818646756, 0.02396377063088417)),
 ('23293894', (0.0009820168240258686, 0.0062363254731424254)),
 ('28763446', (0.002099332159194301, 0.009451628609396915)),
 ('499810', (0.008611841621115186, 0.0426415949385577)),
 ('3681052', (0

In [ ]:
final_rdd = root_rdd.map(lambda x: (x[0],(x[1][0]/(x[1][1]*sum_sqroot))))

In [ ]:
final_rdd.collect()

Out[83]: [('3497365', 0.23835923081669394),
 ('2442464', 0.23835923081669394),
 ('3879783', 0.2894665666993864),
 ('23726874', 0.23835923081669397),
 ('2570318', 0.23835923081669394),
 ('2650227', 0.2814255698183192),
 ('3759313', 0.23835923081669394),
 ('6106857', 0.2936758595901415),
 ('4288473', 0.29367585959014153),
 ('981884', 0.23835923081669397),
 ('20089961', 0.27688506642027205),
 ('28820585', 0.2383592308166939),
 ('26553611', 0.23835923081669394),
 ('23293894', 0.29367585959014153),
 ('28763446', 0.4142405442995539),
 ('499810', 0.3766523548965602),
 ('3681052', 0.23835923081669394),
 ('963021', 0.29367585959014153),
 ('2642113', 0.2383592308166939),
 ('161334', 0.3467900713629504),
 ('7920068', 0.29367585959014153),
 ('28105466', 0.4031914560035744),
 ('403227', 0.23835923081669394),
 ('4192073', 0.23835923081669394),
 ('788543', 0.23835923081669394),
 ('15351941', 0.29025396238774903),
 ('15234055', 0.23835923081669394),
 ('461322', 0.23835923081669394),
 ('31623179', 0.23

In [ ]:
final_rdd = final_rdd.sortBy(lambda x:-x[1])
final_rdd.collect()

Out[84]: [('32583135', 0.8996108454779138),
 ('5959715', 0.654060380942401),
 ('1630107', 0.6340371378529063),
 ('12635192', 0.6103873397984452),
 ('1613731', 0.5189677007303077),
 ('13281115', 0.5134323387949475),
 ('686033', 0.5111104864230195),
 ('44302', 0.5072284900144743),
 ('5600552', 0.5054674301335875),
 ('5454803', 0.5048261509196214),
 ('5099214', 0.4955630817304639),
 ('396282', 0.49542012609721825),
 ('9974375', 0.49473298484109235),
 ('2853254', 0.4830267490920662),
 ('1795394', 0.47803976858307473),
 ('19308640', 0.4718319163290334),
 ('1006385', 0.46960444302074167),
 ('2045991', 0.469051173537838),
 ('17752771', 0.4657322258789135),
 ('161901', 0.4635245194720279),
 ('15107077', 0.4614778663250862),
 ('3117881', 0.4605062742651046),
 ('26499456', 0.4592497088051349),
 ('33773022', 0.45333461923533724),
 ('3541389', 0.45062846096363035),
 ('2051753', 0.44777928338018863),
 ('22726015', 0.4460517006980093),
 ('825149', 0.44198672308003395),
 ('587507', 0.4419867230800339

In [ ]:
result = final_rdd.join(movies_metadata)

In [ ]:
result.collect()

Out[86]: [('2220694', (0.4284056880078409, "can't stop the music")),
 ('9656385', (0.4220442889707865, 'lamb')),
 ('4607218', (0.41061409150967026, 'rose red')),
 ('20333594', (0.40011421282753934, 'ride beyond vengeance')),
 ('1389099', (0.3907116066447565, 'under the tuscan sun')),
 ('28621291', (0.37406907168896614, 'barbie: a fashion fairy tale')),
 ('9251956', (0.37394456901449724, 'sunny side up')),
 ('1335244', (0.3722057770420828, 'electric dreams')),
 ('10614861', (0.36676643620924476, 'joroo ka ghulam')),
 ('2421509', (0.3667664362092447, 'another woman')),
 ('2354145', (0.35597234527691285, 'curse of the pink panther')),
 ('3410910', (0.3559723452769128, 'night after night')),
 ('2478900', (0.35269470766336986, 'house of games')),
 ('25898800', (0.34965269835761986, 'a hole in one')),
 ('4499436', (0.34922160752461057, 'way down east')),
 ('30474049', (0.34171707640243365, 'ticking clock')),
 ('3024499', (0.33913847018125537, 'limbo')),
 ('3022675', (0.33879212745585563, 'th

In [ ]:
result.take(10)

Out[87]: [('2220694', (0.4284056880078409, "can't stop the music")),
 ('9656385', (0.4220442889707865, 'lamb')),
 ('4607218', (0.41061409150967026, 'rose red')),
 ('20333594', (0.40011421282753934, 'ride beyond vengeance')),
 ('1389099', (0.3907116066447565, 'under the tuscan sun')),
 ('28621291', (0.37406907168896614, 'barbie: a fashion fairy tale')),
 ('9251956', (0.37394456901449724, 'sunny side up')),
 ('1335244', (0.3722057770420828, 'electric dreams')),
 ('10614861', (0.36676643620924476, 'joroo ka ghulam')),
 ('2421509', (0.3667664362092447, 'another woman'))]

In [ ]:
movies = result.map(lambda x: (x[1][1]))

In [ ]:
movies.take(10)

Out[89]: ["can't stop the music",
 'lamb',
 'rose red',
 'ride beyond vengeance',
 'under the tuscan sun',
 'barbie: a fashion fairy tale',
 'sunny side up',
 'electric dreams',
 'joroo ka ghulam',
 'another woman']